# Piperack Generator

## Module imports and connection to STAAD

In [1]:
from base.helper.general import *
from base.geometry_base.rectangle import *
from base.staad_base.geometry import *
from base.structural_elements.beam import *
from base.structural_elements.column import *
from base.structural_elements.brace import *
from base.piperack.portal import *
from base.piperack.piperack import *
from base.piperack.tier import *
from base.staad_base.load import *
from base.staad_base.design import *
from base.staad_base.property import *
from output.md_output import *

from base.staad_base.optimise_member import *
from IPython.display import display, Markdown
from pyperclip import copy

openSTAAD,STAAD_objects = get_openSTAAD()

geometry = STAAD_objects.geometry
property = STAAD_objects.property
output = STAAD_objects.output
support = STAAD_objects.support
load = STAAD_objects.load

add_beams = add_beams(geometry=geometry)
assign_profile = assign_profile(property=property)
assign_specification = assign_specification(property=property)

## Inputs

### Steel

In [2]:
base_point_of_first_portal = Point3D(0,0,0)
width_of_piperack = 8
portal_distances = [0,8,16,24,30,36,42]
column_distances = [0,width_of_piperack]
tier_elevations = [3,6,9,12,14]
long_beam_elevations = [4.5,7.5,10.5,tier_elevations[-1]]
braces_placement = [False,True,False,False,True,False]
brace_pattern = BracePattern.X_Pattern or BracePattern.Tent     # To be implemented

### Concrete

In [3]:
pedestal_height = 2
pedestal_offset = 1

## Generator Code

### Profiles

In [4]:
concrete_profile_list = {
    PiperackMembers.Pedestals : Rectangle(1.2,0.8)
}

steel_profile_list = {
    PiperackMembers.Columns : 'ISMB600',
    PiperackMembers.TierBeams : 'ISMB500',
    PiperackMembers.LongitudinalBeams : 'ISMB600',
    PiperackMembers.PlanBracing : 'ISA100x100x6',
    PiperackMembers.VerticalBracing : 'ISA150x150x10',
    PiperackMembers.Stubs : 'ISMB500',
    PiperackMembers.IntermediateTransverseBeams : 'ISMB500',
    PiperackMembers.IntermediateLongitudinalBeams : 'ISMB400'
}

profile_ids = {}

for member_type,profile in concrete_profile_list.items():
    if ((isinstance(profile, str) and profile.isdigit()) or isinstance(profile, (int, float))):
        profile_ids[member_type] = int(profile)
    else:
        profile_ids[member_type] = create_concrete_beam_property(property,profile)

for member_type,profile in steel_profile_list.items():
    if ((isinstance(profile, str) and profile.isdigit()) or isinstance(profile, (int, float))):
        profile_ids[member_type] = int(profile)
    else:
        profile_ids[member_type] = create_steel_beam_property(property,10,profile,0,0,0)
        
display(Markdown(create_profile_markdown_table(concrete_profile_list, steel_profile_list, profile_ids)))

## Profile List

| Member Type | Profile | Profile ID |
|-------------|---------|------------|
| 2 | Rectangle : length = 1.2 and width = 0.8 | 92 |
| 1 | ISMB600 | 93 |
| 0 | ISMB500 | 94 |
| 3 | ISMB600 | 93 |
| 5 | ISA100x100x6 | 95 |
| 4 | ISA150x150x10 | 96 |
| 6 | ISMB500 | 94 |
| 7 | ISMB500 | 94 |
| 8 | ISMB400 | 97 |


### Supports

In [5]:
support_id = support.CreateSupportFixed()

### Specifications

#### Releases

In [6]:
start_release_spec = property.CreateMemberReleaseSpec(0,set_DOFReleaseArray())
end_release_spec = property.CreateMemberReleaseSpec(1,set_DOFReleaseArray())
set_start_end_release = get_start_end_release_function(property=property,start_release_spec=start_release_spec,end_release_spec=end_release_spec)

#### Truss and Offset

In [ ]:
truss_spec = property.CreateMemberTrussSpec()
member_offset_spec = property.CreateMemberOffsetSpec(0,0,0,1,0)

display(Markdown(create_spec_markdown_table(start_release_spec,end_release_spec,truss_spec,member_offset_spec)))

'## Member Specifications\n\n| Specification Type | Details |\n|-------------------|---------|\n| Start Release | 0 |\n| End Release | 1 |\n| Truss | 2 |\n| Offset Member | 3 |\n'

## Member Creation

### Primary Members

In [8]:
portals = []
longitudinal_beams = []
stubs = []
intermediate_transverse_beams = []
intermediate_long_beams = []
plan_braces = []
vertical_braces = []

portal_beam_ids = [] 
portal_column_ids = [] 
portal_pedestal_ids = [] 
support_node_ids = []
long_beam_ids = [] 
stub_ids = []
intermediate_transverse_ids = []
intermediate_long_ids = []
plan_brace_ids = []
vertical_brace_ids = []

portal_count = len(portal_distances)
portal : PiperackPortal = PiperackPortal(base=base_point_of_first_portal) 

for column in column_distances:
    support_point = Point3D(column,0,0)-Point3D(0,pedestal_height,0)
    portal.add_pedestal(Column3D(base=support_point,height=pedestal_height))
    portal.add_column(Column3D(base=Point3D(column,0,0),height=tier_elevations[-1])) 
    
for tier in tier_elevations:
    for column_i in range(len(column_distances)-1):
        portal.add_beam(Beam3D(start=Point3D(column_distances[column_i],tier,0),end=Point3D(column_distances[column_i+1],tier,0))) 

portals = [portal.shift(Point3D(0,0,dist)) for dist in portal_distances]

for portal_i in range(len(portals)-1):
    for column in column_distances:
        for long_beam in long_beam_elevations:
            long_beam_x = Beam3D(start=Point3D(column,long_beam,portals[portal_i].base.z),end=Point3D(column,long_beam,portals[portal_i+1].base.z))
            longitudinal_beams.append(long_beam_x)

    
for portal in portals:
    portal_beam_ids = [*portal_beam_ids,*add_beams(portal.beams)]
    portal_column_ids = [*portal_column_ids,*add_beams(portal.columns)]
    portal_pedestal_ids = [*portal_pedestal_ids,*add_beams(portal.pedestals)]
    
    portal_tier_assign = assign_profile(portal_beam_ids,profile_ids[PiperackMembers.TierBeams])
    assign_profile(portal_column_ids,profile_ids[PiperackMembers.Columns])
    assign_profile(portal_pedestal_ids,profile_ids[PiperackMembers.Pedestals])

long_beam_ids = add_beams(longitudinal_beams)
long_release = set_start_end_release(long_beam_ids)

for portal in portals:
    for pedestal in portal.pedestals:
        support_node_ids.append(add_support_node(geometry,support,pedestal.start,support_id))

assign_specification(portal_pedestal_ids,member_offset_spec)
        
long_assign = assign_profile(long_beam_ids,profile_ids[PiperackMembers.LongitudinalBeams])

markdown_output = ''
markdown_output = "## Primary Members Created\n\n"
markdown_output += f"| Member Types     | IDs                  | Profile Assignment | Release Assignmnet |\n"
markdown_output += f"| ---              | ---                  | --- | --- |\n"
markdown_output += f"| Portal Columns   | {portal_column_ids}   | {portal_tier_assign} | \n"
markdown_output += f"| Portal Pedestals | {portal_pedestal_ids} | \n"
markdown_output += f"| Portal Beams     | {portal_beam_ids}     |\n"
markdown_output += f"| Long Beams       | {long_beam_ids}       | {long_assign} | {long_release} | \n"
markdown_output += f"| Support Nodes    | {support_node_ids}    |\n"

display(Markdown(markdown_output))

## Primary Members Created

| Member Types     | IDs                  | Profile Assignment | Release Assignmnet |
| ---              | ---                  | --- | --- |
| Portal Columns   | [450, 451, 457, 458, 464, 465, 471, 472, 478, 479, 485, 486, 492, 493]   | [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True] | 
| Portal Pedestals | [8, 9, 17, 18, 26, 27, 35, 36, 44, 45, 53, 54, 62, 63] | 
| Portal Beams     | [445, 446, 447, 448, 449, 452, 453, 454, 455, 456, 459, 460, 461, 462, 463, 466, 467, 468, 469, 470, 473, 474, 475, 476, 477, 480, 481, 482, 483, 484, 487, 488, 489, 490, 491]     |
| Long Beams       | [494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541]       | [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True] | [(True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True)] | 
| Support Nodes    | [13, 14, 27, 28, 41, 42, 55, 56, 69, 70, 83, 84, 97, 98]    |


### Secondary Members

In [9]:
long_beams_y = group_beams_by_y(longitudinal_beams)

previous_y = None
for y,group_y in long_beams_y.items():
    base = portal.base.y if not previous_y else previous_y
    long_beams_sorted = beams_sorted_yxz(group_y)

    for beam_i in range(len(long_beams_sorted)-1):
        portal_i = beam_i%(len(portals)-1)
        if(braces_placement[portal_i]):
            
            long_beam_i : Beam3D = long_beams_sorted[beam_i]

            vb_1 = Beam3D(  start=Point3D(long_beam_i.start.x,base,long_beam_i.start.z),
                            end=long_beam_i.mid())
            vb_2 = Beam3D(  start=Point3D(long_beam_i.start.x,base,long_beam_i.end.z),
                            end=long_beam_i.mid())
                
            vertical_braces.append(vb_1)
            vertical_braces.append(vb_2)

    previous_y = y

vertical_brace_ids = add_beams(vertical_braces)

assign_specification(vertical_brace_ids,truss_spec)
assign_profile(vertical_brace_ids,profile_ids[PiperackMembers.VerticalBracing])

create_stub = lambda beamA,beamB : Beam3D(start=beamA.mid(),end=beamB.mid())

for long_beam_i in range(len(longitudinal_beams)-1):
    long_ii:Beam3D = longitudinal_beams[long_beam_i]
    long_ij:Beam3D = longitudinal_beams[long_beam_i+1]
    if(long_ii.start.eq_x(long_ij.start) and long_ii.end.eq_z(long_ij.end)):
        stubs.append(create_stub(long_ii,long_ij))

stubs = beams_sorted_yzx(stubs)

for stub_i in range(0, len(stubs), 2):
    if stub_i + 1 < len(stubs):  
        stub1, stub2 = stubs[stub_i], stubs[stub_i + 1]
        stub1_mid,stub2_mid = stub1.mid(),stub2.mid()
        intermediate_transverse_beams.append(Beam3D(start=stub1_mid,end=stub2_mid)\
                                                .shift_to_y(closest_to(tier_elevations,stub1_mid.mid(stub2_mid).y)))

stub_ids = add_beams(stubs)
assign_profile(stub_ids,profile_ids[PiperackMembers.Stubs])
set_start_end_release(stub_ids)
    
intermediate_transverse_ids = add_beams(intermediate_transverse_beams)
assign_profile(intermediate_transverse_ids,profile_ids[PiperackMembers.IntermediateTransverseBeams])
set_start_end_release(intermediate_transverse_ids)

portal_beams_y = group_beams_by_y([beam for portal in portals for beam in portal.beams])
for group_y in portal_beams_y.values():
    sorted_beams = beams_sorted_yzx(group_y)
    for beam_i in range(len(sorted_beams)-1):
        beam1 : Beam3D = sorted_beams[beam_i]
        beam2 : Beam3D = sorted_beams[beam_i+1]
        intermediate_long_beams.append(Beam3D(start=beam1.mid(),end=beam2.mid()))

        if(braces_placement[beam_i]):
            mid_pt = beam1.start.mid(beam2.end)
            for pt_x in [beam1.start,beam1.end,beam2.start,beam2.end]:
                plan_braces.append(Beam3D(start=pt_x,end=mid_pt))


intermediate_long_ids = add_beams(intermediate_long_beams)
assign_profile(intermediate_long_ids,profile_ids[PiperackMembers.IntermediateLongitudinalBeams])
set_start_end_release(intermediate_long_ids)

plan_brace_ids = add_beams(plan_braces)
assign_specification(plan_brace_ids,truss_spec)
assign_profile(plan_brace_ids,profile_ids[PiperackMembers.PlanBracing])
    

markdown_output = ''
markdown_output = "## Secondary Members Created\n\n"
markdown_output += "| Member Types | IDs | \n"
markdown_output += "| --- | --- |\n"
markdown_output += f"| Stubs | {stub_ids} |\n"
markdown_output += f"| Intermediate Trans Beams | {intermediate_transverse_ids} |\n"
markdown_output += f"| Intermediate Long Beams | {intermediate_long_ids} |\n"
markdown_output += f"| Plan Braces | {plan_brace_ids} |\n"
markdown_output += f"| Vertical Braces | {vertical_brace_ids} |\n"

display(Markdown(markdown_output))

## Secondary Members Created

| Member Types | IDs | 
| --- | --- |
| Stubs | [542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577] |
| Intermediate Trans Beams | [272, 578, 278, 281, 579, 287, 290, 580, 296, 299, 581, 305, 308, 582, 314, 317, 583, 323] |
| Intermediate Long Beams | [326, 584, 330, 332, 585, 336, 339, 586, 343, 345, 587, 349, 352, 588, 356, 358, 589, 362, 365, 590, 369, 371, 591, 375, 378, 592, 382, 384, 593, 388] |
| Plan Braces | [390, 391, 392, 393, 395, 396, 397, 398, 401, 402, 403, 404, 407, 408, 409, 410, 413, 414, 415, 416, 419, 420, 421, 422, 425, 426, 427, 428, 431, 432, 433, 434, 436, 437, 438, 439, 441, 442, 443, 444] |
| Vertical Braces | [155, 156, 158, 159, 161, 162, 164, 165, 167, 168, 170, 171, 173, 174, 176, 177, 179, 180, 182, 183, 185, 186, 188, 189, 191, 192, 194, 195, 197, 198, 200, 201] |


## Loads

### Initialization


In [10]:
add_conc_forces_to_members = add_conc_forces_to_members(load)

### Portal Beams

In [11]:
nodes = get_node_incidences(geometry=geometry)
beam_objects:dict = get_beam_objects(geometry=geometry,property=None,nodes=nodes)
beam_nos:list = list(beam_objects.keys())